In [9]:
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.layouts import column,row
output_notebook(hide_banner=True)
from cocoa import covid19 as co
from cocoa import display as cd
from cocoa import world as cw
from cocoa.geo import GeoManager as gm
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.palettes import brewer
import geopandas as gpd
import importlib
w=cw.WorldInfo()

In [4]:
owid = co.DataBase("owid")
coco = cd.CocoDisplay()

In [29]:
g=gm() # Create the instance of GeoManager 
loc= ['France','Spain','Italy','Greece','Belgium','Germany','Austria','Israel']
country_list_of_world=g.to_standard('World',db='owid',interpret_region=True)
a=owid.get_stats(location=loc,which='new_cases')
fig = coco.cocoa_basic_plot([a],['cumul'])
show(fig)
print(a)

    location       date  new_cases     cumul    diff
0     France 2019-12-31        0.0       0.0     0.0
1     France 2020-01-01        0.0       0.0     0.0
2     France 2020-01-02        0.0       0.0     0.0
3     France 2020-01-03        0.0       0.0     0.0
4     France 2020-01-04        0.0       0.0     0.0
..       ...        ...        ...       ...     ...
284   Israel 2020-10-10     1465.0  287858.0 -3447.0
285   Israel 2020-10-11     2145.0  290003.0   680.0
286   Israel 2020-10-12     1825.0  291828.0  -320.0
287   Israel 2020-10-13     3209.0  295037.0  1384.0
288   Israel 2020-10-14     2237.0  297274.0  -972.0

[2312 rows x 5 columns]


In [30]:
g.get_standard()

'iso2'

In [18]:
geojsondatafile = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'

gdf=gpd.read_file(geojsondatafile)
gdf.columns = ['country_code', 'location', 'geometry']
gdf.head()

,country_code,location,geometry
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."


In [27]:
print(a)

    location       date  new_cases     cumul    diff
0     France 2019-12-31        0.0       0.0     0.0
1     France 2020-01-01        0.0       0.0     0.0
2     France 2020-01-02        0.0       0.0     0.0
3     France 2020-01-03        0.0       0.0     0.0
4     France 2020-01-04        0.0       0.0     0.0
..       ...        ...        ...       ...     ...
284   Israel 2020-10-10     1465.0  287858.0 -3447.0
285   Israel 2020-10-11     2145.0  290003.0   680.0
286   Israel 2020-10-12     1825.0  291828.0  -320.0
287   Israel 2020-10-13     3209.0  295037.0  1384.0
288   Israel 2020-10-14     2237.0  297274.0  -972.0

[2312 rows x 5 columns]


In [28]:
merged = gdf.merge(a, on ='location', how = 'left')
merged=merged#[["country","geometry","evol0","evol1","acc","cases"]]
merged.head()

,country_code,location,geometry,date,new_cases,cumul,diff
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",NaT,NaN,NaN,NaN
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6...",NaT,NaN,NaN,NaN
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",NaT,NaN,NaN,NaN
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",NaT,NaN,NaN,NaN
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",NaT,NaN,NaN,NaN


In [25]:
import json
#Read data to json
merged_json = json.loads(merged.to_json())

#Convert to str like object
json_data = json.dumps(merged_json)

TypeError: Object of type Timestamp is not JSON serializable

In [23]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.palettes import brewer
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['RdYlGn'][10] # see https://docs.bokeh.org/en/latest/docs/reference/palettes.html

#Reverse color order so that dark blue is highest obesity.

hover = HoverTool(tooltips = [ ('Country','@country'),
                              ('Total '+whichdata, '@cases'),
                              ('Evolution last week in %','@evol0'),
                              ('Evolution previous week in %','@evol1') ] )

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = -50, high=50)
#Define custom tick labels for color bar.
#tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%'}#, '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')#, major_label_overrides = tick_labels)

#Create figure object.
f = figure(title = 'Acceleration (or deceleration) of weekly '+whichdata+' in % at '+str(lastdate), plot_height = 700 , plot_width = 950,tools = [hover])#, toolbar_location = None)
f.xgrid.grid_line_color = None
f.ygrid.grid_line_color = None

#Add patch renderer to figure. 
f.patches('xs','ys', source = geosource,fill_color = {'field' :'acc', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
f.add_layout(color_bar, 'below')

#Display figure inline in Jupyter Notebook.
output_notebook()

NameError: name 'json_data' is not defined

In [12]:
#dCountryCode=pd.read_html('https://www.iban.com/country-codes')[0]
#dCountryCode.head()
#dCountryDataMerge = dCountryData.merge(dCountryCode, left_on = 'CountryCode', right_on = 'Alpha-2 code', how = 'left')
#merged.fillna('No data', inplace = True)


dCountryCode=pd.read_html('https://www.iban.com/country-codes')[0]
dCountryCode.head()



geojsondatafile = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'

gdf=gpd.read_file(geojsondatafile)
gdf.columns = ['country_code', 'country', 'geometry']
gdf.head()
merged = gdf.merge(dCountryDataMerge, left_on = 'country_code', right_on = 'Alpha-3 code', how = 'left')
merged.sort_values('country_code').head()
merged.head()
#Read data to json
merged_json = json.loads(merged.to_json())

#Convert to str like object
json_data = json.dumps(merged_json)



#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

hover = HoverTool(tooltips = [ ('Country','@country (@country_code / @CountryCode)'),
                              ('Total deaths', '@TotalDeaths'),
                              ('New deaths last day','@NewDeaths')])

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high=500)
#Define custom tick labels for color bar.
#tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%'}#, '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')#, major_label_overrides = tick_labels)

#Create figure object.
p = figure(title = 'New deaths over the world at '+thedate, plot_height = 600 , plot_width = 950,tools = [hover])#, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'NewDeaths', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')

#Display figure inline in Jupyter Notebook.
output_notebook()

#Display figure.
show(p)

NameError: name 'dCountryDataMerge' is not defined